In [35]:
!python -V

Python 3.8.10


In [36]:
import pandas as pd

In [37]:
import pickle

In [38]:
import seaborn as sns
import matplotlib.pyplot as plt

In [39]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [40]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    print(f"{filename} : {df.index}")

    df["duration"] = df.dropOff_datetime - df.pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    print(f'{filename} : {df["duration"].mean()}')    

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    print(f'{filename} : {df["PUlocationID"].isnull().sum() * 100 / len(df)}')

    df[["PUlocationID", "DOlocationID"]] = df[["PUlocationID", "DOlocationID"]].fillna(-1)

    categorical = ["PUlocationID", "DOlocationID"]
    df[categorical] = df[categorical].astype(str)    

    return df

In [41]:
df_train = read_dataframe('../data/fhv_tripdata_2021-01.parquet')
df_val = read_dataframe('../data/fhv_tripdata_2021-02.parquet')

../data/fhv_tripdata_2021-01.parquet : RangeIndex(start=0, stop=1154112, step=1)
../data/fhv_tripdata_2021-01.parquet : 19.167224093791006
../data/fhv_tripdata_2021-01.parquet : 83.52732770722618
../data/fhv_tripdata_2021-02.parquet : RangeIndex(start=0, stop=1037692, step=1)
../data/fhv_tripdata_2021-02.parquet : 20.706986225199763
../data/fhv_tripdata_2021-02.parquet : 85.71354986754037


In [42]:
categorical = ['PUlocationID', 'DOlocationID']

dv = DictVectorizer()

train_dicts = df_train[categorical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [43]:
X_train

<1109826x525 sparse matrix of type '<class 'numpy.float64'>'
	with 2219652 stored elements in Compressed Sparse Row format>

In [44]:
target = 'duration'
y_train = df_train[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

10.528519390019362

In [45]:
y_val = df_val[target].values

y_pred_val = lr.predict(X_val)

mean_squared_error(y_val, y_pred_val, squared=False)

11.014285972895173